In [1]:
import cv2
import numpy as np

In [ ]:
cap = cv2.VideoCapture(0)
#bir tane frame oku
ret,frame = cap.read()

if ret==False:
    print("Uyari")

#deteciton
face_cascade = cv2.CascadeClassifier("media/haarcascade_frontalface_default.xml") #xml dosyasini oku
face_rects =face_cascade.detectMultiScale(frame) #yuz tespiti yap


(face_x,face_y,w,h) = tuple(face_rects[0])  #yuzun koordinatlarini al

track_window = (face_x,face_y,w,h) #takip penceresi meanashift algoritmasi girdisi

#region of interest (roi) al

roi = frame[face_y:face_y+h,face_x:face_x+w] #yuzun oldugu alani al

hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV) #hsv formatina cevir

roi_hist=cv2.calcHist([hsv_roi],[0],None,[180],[0,180]) #takip icin histogram hesapla

cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX) #normalizasyon

#takip icin gerekli durdurma kriterleri
#count: hesaplanacak maksimum oge sayisi
#epsilon : degisiklik

term_crit= (cv2.TERM_CRITERIA_EPS| cv2.TERM_CRITERIA_COUNT,5,1) #durma kriterleri

while True:
    ret,frame = cap.read() #kameradan frame oku
    frame = cv2.flip(frame,1) #goruntuyu dondur
    if ret:
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

        #histogrami bir goruntude bulmak icin backprojection kullanilir
        #backprojection: bir goruntudeki her pikselin histograma gore olasiligini hesaplar

        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1) #backprojection hesapla

        ret , track_window = cv2.meanShift(dst,track_window,term_crit) #mean shift algoritmasi

        (x,y,w,h) = track_window #takip penceresinin koordinatlari

        img2 = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),5) #takip penceresini ciz

        cv2.imshow("MeanShift Tracking",img2)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()



















